In [10]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy import sparse
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

rc('animation', html='jshtml')

# Initial params
dz = 0.001
z_string = np.arange(-1, 1, dz) # string, 2m long
arr_dim = z_string.size
v_p = 400 # Wavespeed [m/s] = sqrt(T / mu)
b = .01 # impact parameter, 1 cm away
v_0 = b / (b**2 + z_string**2) # initial velocity as function of z
y_0 = np.zeros(arr_dim) # initial position as function of z

# Laplace operator with finite differences
laplacian = sparse.diags([1, -2, 1], [-1, 0, 1], shape= (arr_dim, arr_dim)) / dz**2

def couple_fun(t, u):
    """" 
    Function that returns first derivative in time => u is coupled position and velocity =>
        First N points return velocity: second N points of u
        Second N points return accelleration: v_p^2 * Laplacian.[first N points of u]
    """
    du = np.zeros(2 * arr_dim) 
    du[:arr_dim] = u[arr_dim:] # velocity is second half of coupled array (u)
    du[arr_dim:] = v_p**2 * laplacian.dot(u[:arr_dim]) # acceleration is wave eq on first half of coupled array (u)
    # fixed ends?
    return du

# Initialize coupled array
u_0 = np.zeros(2*arr_dim)
u_0[:arr_dim] = y_0
u_0[arr_dim:] = v_0

# time array
dt = 0.00001
t0 = 0.0
tf = 0.01
t_eval = np.arange(t0, tf, dt)

sol = solve_ivp(couple_fun, t_span= [t0, tf], y0= u_0, t_eval= t_eval, method="RK23")

fig1, ax1 = plt.subplots()
ax1.set_xlabel('String position [m]')
title = ax1.set_title(' ')
line1, = ax1.plot([], [], label='Position')
line2, = ax1.plot([], [], label='Velocity')
ax1.legend()

def init():
    line1.set_data([],[])
    return line1

def animate(i):
    line1.set_data(z_string, sol.y[:arr_dim, i])
    line2.set_data(z_string, sol.y[arr_dim:, i])
    title.set_text('Time = {:.2f} ms'.format(1000*sol.t[i]))
    return line1, line2

anim = animation.FuncAnimation(fig1, animation, init_func=init, frames=len(sol.t), interval=5, blit=True)

HTML(anim.to_jshtml())
# fig1.tight_layout()
# fig1.show()





<IPython.core.display.Javascript object>

TypeError: 'module' object is not callable